In [13]:
from huggingface_hub import snapshot_download
from glob import glob
from streaming import MDSWriter, LocalDataset
import json
import os

In [2]:
while True:
    try:
        folder = snapshot_download(
            repo_id="mesolitica/malaysian-stt", 
            local_dir = './malaysian-stt',
            repo_type = 'dataset',
                 max_workers = 10)
        break
    except Exception as e:
        print(e)
        pass

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/2.70k [00:00<?, ?B/s]

tamil-stt.jsonl:   0%|          | 0.00/701M [00:00<?, ?B/s]

science-en-stt.jsonl:   0%|          | 0.00/581M [00:00<?, ?B/s]

mandarin-stt.jsonl:   0%|          | 0.00/287M [00:00<?, ?B/s]

science-ms-stt.jsonl:   0%|          | 0.00/514M [00:00<?, ?B/s]

malaysian-stt.jsonl:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

In [4]:
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-imda-large-v3-timestamp/resolve/main/prepared-imda-ms.jsonl
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-imda-large-v3-timestamp/resolve/main/prepared-imda.jsonl

In [12]:
# !wget https://huggingface.co/datasets/malaysia-ai/malay-conversational-speech-corpus-whisper-format/resolve/main/malay-conversational-speech-corpus-whisper-format.zip
# !wget https://huggingface.co/datasets/malaysia-ai/malay-conversational-speech-corpus-whisper-format/resolve/main/malay-conversational-speech-corpus-whisper-format.json
# !unzip malay-conversational-speech-corpus-whisper-format.zip

In [11]:
with open('malay-conversational-speech-corpus-whisper-format.json') as fopen:
    malay_conversational = json.load(fopen)
    
malay_conversational[0]

{'filename': 'malay-conversational-speech-corpus-whisper-format/0.wav',
 'Y': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> hai weh<|1.24|><|endoftext|>'}

In [16]:
# !wget https://huggingface.co/datasets/malaysia-ai/iban-whisper-format/resolve/main/iban-wav.zip
# !wget https://huggingface.co/datasets/malaysia-ai/iban-whisper-format/resolve/main/iban-dataset.json
# !unzip iban-wav.zip

In [17]:
with open('iban-dataset.json') as fopen:
    iban = json.load(fopen)
    
iban[0]

{'filename': 'iban-wav/ibf_002_001.wav',
 'Y': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Masjid Al-Takwa di Miri udah nerima agih belanja seratus ribu ringgit ari opis Menteri Anegi Teknologi Hijau Enggau Ai kena ngitu pemanah masjid-nya.<|10.58|><|endoftext|>'}

In [19]:
# !wget https://huggingface.co/datasets/malaysia-ai/scripted-malay-daily-use-speech-corpus-whisper-format/resolve/main/scripted-malay-daily-use-speech-corpus-whisper-format.zip
# !wget https://huggingface.co/datasets/malaysia-ai/scripted-malay-daily-use-speech-corpus-whisper-format/resolve/main/scripted-malay-daily-use-speech-corpus-whisper-format.json
# !unzip scripted-malay-daily-use-speech-corpus-whisper-format.zip

In [20]:
with open('scripted-malay-daily-use-speech-corpus-whisper-format.json') as fopen:
    scripted_malay = json.load(fopen)
scripted_malay[0]

{'filename': 'audio-whisper/0.wav',
 'Y': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Dia tidak tahu apa-apa .<|3.56|><|endoftext|>'}

In [22]:
# !wget https://huggingface.co/datasets/malaysia-ai/sarawakmalay-whisper-format/resolve/main/sarawakmalay.zip
# !wget https://huggingface.co/datasets/malaysia-ai/sarawakmalay-whisper-format/resolve/main/dataset.json -O sarawakmalay.json
# !unzip sarawakmalay.zip

In [23]:
with open('sarawakmalay.json') as fopen:
    sarawak = json.load(fopen)
sarawak[0]

{'audio_filename': 'audio/SM_FF_MOBILELEGENDS_002-0.mp3',
 'text': '<|startoftranscript|><|ms|><|transcribe|><|0.54|> : Hi, nama kamek Sofia. Kamek mok temuduga kitak untuk menceritakan permainan yang kitak suka main — sama ada permainan traditional ataupun moden. Boleh sik perkenalkan dirik kitak.<|10.44|><|endoftext|>'}

In [29]:
extra = [
    malay_conversational,
    iban,
    scripted_malay,
    sarawak,
]

In [25]:
filenames = glob('malaysian-stt/*.jsonl') + [
    'prepared-imda-ms.jsonl', 
    'prepared-imda.jsonl',
    'indonesian-stt.jsonl'
]

In [26]:
import os
import json
from tqdm import tqdm

not_in = []

for f in filenames:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            if '/ubuntu/' in l['audio_filename']:
                l['audio_filename'] = l['audio_filename'].split('/ubuntu/')[1]
            if 'imda' in f:
                l['audio_filename'] = l['audio_filename'].replace('IMDA-STT/', '')
            if 'science' in f:
                l['audio_filename'] = os.path.join('pseudolabel-science-large-v3-timestamp', l['audio_filename'])
            
            if not os.path.exists(l['audio_filename']):
                not_in.append(l['audio_filename'])

7664734it [00:44, 171846.04it/s]
1455087it [00:05, 251480.02it/s]
709423it [00:03, 186478.11it/s]
1347342it [00:05, 266147.73it/s]
850638it [00:04, 190591.47it/s]
1217190it [00:04, 249723.56it/s]
1861125it [00:07, 251767.86it/s]
943282it [00:03, 278630.34it/s]


In [37]:
len(not_in)

0

In [53]:
!rm -rf mosaic-stt

In [54]:
columns = {
    'audio_filename': 'str',
    'text': 'str',
}
hashes = 'sha1', 'xxh64'

not_in = []

with MDSWriter(out='mosaic-stt', columns=columns, compression=None, hashes=hashes) as out:
    for i in range(len(extra)):
        for d in extra[i]:
            if 'audio_filename' in d:
                audio_filename = d['audio_filename']
            else:
                audio_filename = d['filename']
            if 'text' in d:
                text = d['text']
            else:
                text = d['Y']
            l = {
                'audio_filename': audio_filename,
                'text': text,
            }
            if not os.path.exists(l['audio_filename']):
                continue
            
            out.write(l)
            
    for f in filenames:
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                if '/ubuntu/' in l['audio_filename']:
                    l['audio_filename'] = l['audio_filename'].split('/ubuntu/')[1]
                if 'imda' in f:
                    l['audio_filename'] = l['audio_filename'].replace('IMDA-STT/', '')
                if 'science' in f:
                    l['audio_filename'] = os.path.join('pseudolabel-science-large-v3-timestamp', l['audio_filename'])
                if 'new_text' in l:
                    l['text'] = l.pop('new_text')
                    
                if not os.path.exists(l['audio_filename']):
                    not_in.append(l['audio_filename'])
                    continue
                
                l.pop('index', None)
                if len(l.keys()) > 2:
                    print(l)
#                 if len(l.keys()) == 2:
#                     print(l)
                out.write(l)

7664734it [01:00, 126997.63it/s]
1455087it [00:11, 130649.14it/s]
709423it [00:07, 96981.72it/s] 
1347342it [00:08, 166589.94it/s]
850638it [00:08, 98647.26it/s] 
1217190it [00:08, 141937.91it/s]
1861125it [00:13, 136714.88it/s]
943282it [00:07, 127295.11it/s]


In [55]:
dataset = LocalDataset('mosaic-stt')
len(dataset)

16056567

In [56]:
import IPython.display as ipd
ipd.Audio(dataset[0]['audio_filename'])

In [57]:
dataset[0]

{'audio_filename': 'malay-conversational-speech-corpus-whisper-format/0.wav',
 'text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> hai weh<|1.24|><|endoftext|>'}